# Python-1, Лекция 6

Лектор: Хайбулин Даниэль

Подготовил материал: Хайбулин Даниэль

Сегодня мы продолжим говорить о классах.

## Методы классов

В классах можно делать "специальные пометки" (на самом деле это декораторы, о них поговорим позже) над методом: **статичные методы** и **методы класса**.

### Static method

<div style="
    background-color: #44944A;
    padding: 15px;
    border-radius: 10px;
    border: 1px solid #fbfbfbff;
    text-align: center;
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 15px;
">
    <span style="color: white; font-weight: bold;">
        Статичный метод - метод, привязанный к классу.
    </span>
</div>

In [ ]:
class SampleClass:
    @staticmethod
    def example() -> None:
        print(...)

In [ ]:
assert "example" not in SampleClass().__dict__
assert "example" in SampleClass.__dict__

Заметьте, что статичный метод принадлежит классу, но не принадлежит его инстансу. Главное отличие от ранее изученных обычных методов - это отсутствие аргумента `self`, то есть инстанса класса.

Зачем вообще нужны такие методы? В основном для реализации специфичных конструкторов, отличающихся от основного конструктора - `__init__`. Это называется **фабрика** и, возможно, кто-то разберет это позже на семинарах 12ой лекции когда мы будем говорить об ООП, а пока знакомимся что такое вот есть.

In [ ]:
from typing import Self


class Date:
    def __init__(self, day: int, month: int, year: int) -> Self:
        self.day = day
        self.month = month
        self.year = year
        self.from_american_format("")

    def __str__(self) -> str:
        return f"{self.day:02d}.{self.month:02d}.{self.year}"

    @staticmethod
    def from_iso_format(iso_string: str) -> Self:
        year, month, day = map(int, iso_string.split("-"))
        return Date(day, month, year)

    @staticmethod
    def from_american_format(us_string: str) -> Self:
        month, day, year = map(int, us_string.split("/"))
        return Date(day, month, year)


Теперь можем конструировать инстансы класса даты тремя разными способами: переопределенным `__init__` и двумя статичными методами. Это добавляет нам гибкости в способах создания инстанса класса.

In [ ]:
date1 = Date.from_iso_format("2023-12-25")
print(date1)

date2 = Date.from_american_format("12/25/2023")
print(date2)

assert isinstance(date1, Date)
assert isinstance(date2, Date)

## Наследование

### object - прародитель

<div style="
    background-color: #44944A;
    padding: 15px;
    border-radius: 10px;
    border: 1px solid #fbfbfbff;
    text-align: center;
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 15px;
">
    <span style="color: white; font-weight: bold;">
        <strong>object</strong> - метакласс, от которого наследуются все остальные объекты в питоне.
    </span>
</div>

**Наследование** - это как в генетике, родительский класс передает свои методы, поля, атрибуты (гены) своим детям. Ребенок может наследоваться от множества родителей, при этом порядок наследования методов определяется при помощи алгоритма Method Resolution Order (mro).

In [ ]:
object, object.mro(), {object: None}

Всякий класс наследуется от объекта, отсюда и наличие по дефолту очень многих методов даже у пустых классов.

In [ ]:
dir(object)

In [ ]:
list.mro(), int.mro(), dict.mro()

In [ ]:
class SampleClass: ...

In [ ]:
SampleClass.mro(), dir(SampleClass)

### Наследование методов

Выше мы могли заметить, что от родительского класса `object` базовыми типами наследуется множество dunder методов. Давайте рассмотрим что вообще такое наследование методов:

In [ ]:
class SampleClass:
    def __init__(self, name: str) -> Self:
        self.name = name

        print(f"{self.name} родился!")


class SampleClassChild(SampleClass):
    pass

In [ ]:
SampleClassChild(name="Ребенок")

У класса `SampleClassChild` не определен dunder метод `__init__`, однако мы получили сообщение, которое определено у класса `SampleClass`. Это и есть наследование - если у текущего класса нет метода, то мы идем искать метод в его родительских классах и вызываем его!

### Переопределение методов

Вообще, возникает логичный вопрос: а зачем нам наследование? Действительно, некоторые языки вполне себе обходятся без наследования. Но мы не можем размышлять бинарным подходом - хорошо или плохо. Наследование - это такой же инструмент, как и многие другие инструменты, которые предлагает нам язык программирования. Но стоит хорошенько подумать прежде чем использовать наследование.

Чаще всего наследование помогает нам определить некий общий интерфейс, а в наследуемых от него классах предлагать различную реализацию интерфейса. Эта концепция будет встречаться вам часто, так как она действительно имеет смысл. О ней мы отдельно поговорим на другой лекции.

А сейчас давайте разберемся как описывать интерфейс в родительском классе и переопределять в наследуемых классах: 


In [ ]:
class SampleClass:
    def __init__(self, name: str) -> Self:
        self.name = name

        print(f"{self.name} родился!")


class SampleClassChild(SampleClass):
    def __init__(self, name: str) -> Self:
        super().__init__(SampleClass)
        self.name = name

        print(f"{self.name} вылупился!")

In [ ]:
SampleClassChild("Птенец")

Итак, мы переопределили dunder метод `__init__` и теперь вызывается метод младшего класса.

Помимо этого, мы можем дополнять метод родительского класса в детских классах:

In [ ]:
class SampleClass:
    def __init__(self, name: str) -> Self:
        self.name = name

        print(f"{self.name} родился!")

    @staticmethod
    def p():
        print(1)


class SampleClassChild(SampleClass):
    def __init__(self, name: str) -> Self:
        SampleClass.__init__(self=self, name=name)  # Вызов метода родительского класса

        print(f"{name} родился еще раз!")

    def p(self):
        print(2)

In [ ]:
a = SampleClassChild("Птенец")
a.p()
SampleClassChild.__dict__

На семинарах предлагается подробнее посмотреть на diamond problem и mro, чтобы была возможность поиграться с наследованием. Также рекомендуется рассмотреть `super`.

### slots

Есть такой механизм как `__slots__` - это кортеж, который заменяет словарь нашего класса. Кортеж занимает меньше места чем словарь, поэтому это разумная оптимизация когда мы создаем очень много инстансов класса с небольшим количеством полей:

In [ ]:
class Point:
    __slots__ = ("x", "y")

In [ ]:
p = Point()

In [ ]:
Point().__slots__  # При этом создаем кортеж слотов

In [ ]:
%pip install memory_profiler

Ниже приведу доказательство того, что слоты занимают меньше памяти нежели словари:

In [ ]:
import sys
from memory_profiler import profile


class RegularUser:
    def __init__(self, user_id, name, email, age):
        self.user_id = user_id
        self.name = name
        self.email = email
        self.age = age


class SlotsUser:
    __slots__ = ("user_id", "name", "email", "age")

    def __init__(self, user_id, name, email, age):
        self.user_id = user_id
        self.name = name
        self.email = email
        self.age = age


def measure_memory_usage(user_class, num_instances):
    instances = [
        user_class(i, f"User{i}", f"user{i}@example.com", 20 + i % 30)
        for i in range(num_instances)
    ]

    total_size = sum(sys.getsizeof(instance) for instance in instances)

    if hasattr(instances[0], "__dict__"):
        dict_size = sum(sys.getsizeof(instance.__dict__) for instance in instances)
        total_size += dict_size

    return total_size


num_objects = 100000

regular_memory = measure_memory_usage(RegularUser, num_objects)

slots_memory = measure_memory_usage(SlotsUser, num_objects)

print(
    f"Память для {num_objects} обычных объектов: {regular_memory / 1024 / 1024:.2f} MB"
)
print(
    f"Память для {num_objects} объектов со slots: {slots_memory / 1024 / 1024:.2f} MB"
)
print(f"Экономия памяти: {(regular_memory - slots_memory) / 1024 / 1024:.2f} MB")
print(f"Процент экономии: {(1 - slots_memory / regular_memory) * 100:.2f}%")


@profile
def create_regular_users():
    return [
        RegularUser(i, f"User{i}", f"user{i}@example.com", 20 + i % 30)
        for i in range(num_objects)
    ]


@profile
def create_slots_users():
    return [
        SlotsUser(i, f"User{i}", f"user{i}@example.com", 20 + i % 30)
        for i in range(num_objects)
    ]


if __name__ == "__main__":
    print("=== Создание обычных объектов ===")
    regular_users = create_regular_users()

    print("=== Создание объектов со slots ===")
    slots_users = create_slots_users()

    print(
        f"\nРазмер одного обычного объекта: {sys.getsizeof(RegularUser(1, 'Test', 'test@example.com', 25)) + sys.getsizeof(RegularUser(1, 'Test', 'test@example.com', 25).__dict__)} байт"
    )
    print(
        f"Размер одного объекта со slots: {sys.getsizeof(SlotsUser(1, 'Test', 'test@example.com', 25))} байт"
    )

## Переопределение dunder методов

Подходим к **магии** питона - определение **dunder** или **магических** методов. Определение таких методов позволяет нам встраивать наш класс в уже существующий функционал языка и пользоваться им на полную.

Тут мы просто посмотрим на примере на базовые магические методы, на следующих лекциях будем рассматривать более серьезные магические методы и чего с ними можно добиться.

In [ ]:
from typing import Self


class A:
    def __init__(self, name: str) -> Self:
        self.name = name

    def __repr__(self) -> str:  #  Этот метод мы используем лишь для дебага
        return f"Object id: {id(self)}, object name: {self.name}"

    def __str__(self) -> str:  #  Этот метод вызывается когда мы делаем print
        return f"Объект класса {self.__class__} по имени {self.name}"


In [ ]:
print(A(name="abc"))

In [ ]:
print(A(name="abc"))

Теперь переопределим арифметику, чтобы научить пользовательские классы в математику.

In [ ]:
from typing import Union, Any


class A:
    def __init__(self, x: int) -> Self:
        self.x = x

    def __add__(self, other: Self) -> Self:
        if isinstance(other, A):
            return A(self.x + other.x)
        if isinstance(other, str):
            return other + str(self.x)
        return NotImplemented

    def __iadd__(self, other: Self) -> Self:
        self.x += other.x
        return self

    def __mul__(self, other: Union[int, Any]) -> Self:
        print("A:mul")
        if type(other) is int:
            return A(self.x * other)
        elif hasattr(other, "x"):
            return A(self.x * other.x)
        else:
            print("Не могу умножить!")

    def __rmul__(self, other: Union[int, Any]) -> Self:
        print("A:rmul")
        if type(other) is int:
            return A(self.x * other)
        elif hasattr(other, "x"):
            return A(self.x * other.x)
        else:
            print("Не могу умножить!")

    def __str__(self) -> str:
        return f"A({self.x})"


class B:
    def __init__(self, x: int) -> Self:
        self.x = x

    def __str__(self) -> str:
        return f"B({self.x})"


In [ ]:
A(5) + "ab"
"ab" + A(5)

In [ ]:
a = A(6)
b = A(4)
id_A = id(a)

a += b
assert id(a) == id_A

a = a + b
assert id(a) != id_A

a.x

<div style="
    background-color: #8B0000;
    padding: 15px;
    border: 2px dashed #ba0606;
    border-radius: 5px;
    text-align: center;
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 20px;
">
<span style="color: white; font-weight: bold;">
        dunder методы с приставкой <strong>i</strong> возвращают self.
    </span>
</div>

In [ ]:
a = A(6)
b = B(2)
print(f"a * 3 = {a * 3}")
print(f"3 * a = {3 * a}")
print(f"a * a = {a * a}")
print(f"a * b = {a * b}")
print(f"b * a = {b * a}")

Видим, что тут независимо того того правое это умножение (`__rmul__`) или левое (`__mul__`) - так или иначе возвращается всегда объект класса A. Чтобы это исправить нужно переопределить умножение и у класса B.

In [ ]:
from typing import Union, Any


class A:
    def __init__(self, x: int) -> Self:
        self.x = x

    def __mul__(self, other: Union[int, Any]) -> Self:
        print("A:mul")
        if type(other) is int:
            return A(self.x * other)
        elif hasattr(other, "x"):
            return A(self.x * other.x)
        else:
            print("Не могу умножить!")

    def __rmul__(self, other: Union[int, Any]) -> Self:
        print("A:rmul")
        if type(other) is int:
            return A(self.x * other)
        elif hasattr(other, "x"):
            return A(self.x * other.x)
        else:
            print("Не могу умножить!")

    def __str__(self) -> str:
        return f"A({self.x})"


class B:
    def __init__(self, x: int) -> Self:
        self.x = x

    def __mul__(self, other: Union[int, Any]) -> Self:
        print("B:mul")
        if type(other) is int:
            return B(self.x * other)
        elif hasattr(other, "x"):
            return B(self.x * other.x)
        else:
            print("Не могу умножить!")

    def __rmul__(self, other: Union[int, Any]) -> Self:
        print("B:rmul")
        if type(other) is int:
            return B(self.x * other)
        elif hasattr(other, "x"):
            return B(self.x * other.x)
        else:
            print("Не могу умножить!")

    def __str__(self) -> str:
        return f"B({self.x})"


In [ ]:
a = A(6)
b = B(2)
print(f"a * 3 = {a * 3}")
print(f"3 * a = {3 * a}")
print(f"a * a = {a * a}")
print(f"a * b = {a * b}")
print(f"b * a = {b * a}")

А вот теперь видим, что в одно из умножений у нас создался новый инстанс класса B.

Давайте теперь создадим `Callable` класс: реализуем магический метод `__call__`:

In [ ]:
class Power:
    def __init__(self, p: float):
        self.p = p

    def __call__(self, a: float) -> float:
        return a**self.p


power = Power(3)
power(4)

На семинаре предлагается рассмотреть `__len__`, магические методы сравнимости класса и побольше поиграться с хешируемостью объекта.

## Property

In [ ]:
help(property)

<div style="
    background-color: #44944A;
    padding: 15px;
    border-radius: 10px;
    border: 1px solid #fbfbfbff;
    text-align: center;
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 15px;
">
    <span style="color: white; font-weight: bold;">
        <strong>property</strong> позволяет нам управлять атрибутом класса.
    </span>
</div>

In [ ]:
from typing import Self


class LineItem:
    def __init__(self, description: str, weight: float, price: int) -> Self:
        self.description = description
        self.weight = weight  # Теперь тут вызовется метод set_weight
        self.price = price

    def subtotal(self) -> float:
        return self.weight * self.price

    def get_weight(self) -> float:
        print("Получаем вес!")
        return self.__weight

    def set_weight(self, value) -> None:
        print("Устанавливаем вес!")
        if value > 0:
            self.__weight = value
        else:
            raise ValueError("value must be > 0")

    weight = property(fget=get_weight, fset=set_weight)

На самом деле чаще property используют как декоратор, но об этом мы поговорим на лекции про декораторы.

In [ ]:
item = LineItem(description="Чемодан", weight=3.4, price=300)

In [ ]:
item.weight

## Контроль доступа

<div style="
    background-color: #44944A;
    padding: 15px;
    border-radius: 10px;
    border: 1px solid #fbfbfbff;
    text-align: center;
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 15px;
">
    <span style="color: white; font-weight: bold;">
        По дефолту все атрибуты публичны.
    </span>
</div>

Но вот если я хочу, чтобы никто не мог получить доступ к какому-то атрибуту моего класса, что тогад нужно сделать?

Для этого есть **private** и **protected** поля!

### Private

В питоне нет как таковой приватности у атрибутов, но люди часто используют _ или __, чтобы указать на приватность атрибута. Давайте разберемся что это и зачем.

<div style="
    background-color: #8B0000;
    padding: 15px;
    border: 2px dashed #ba0606;
    border-radius: 5px;
    text-align: center;
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 20px;
">
<span style="color: white; font-weight: bold;">
        __ не обеспечивает приватность атрибута!
    </span>
</div>

Сперва развеем миф о том, что двойное подчеркивание у атрибута обеспечивает его приватность.

In [ ]:
class SampleClass:
    __slots__ = "__name"

    def __init__(self, name: str) -> Self:
        self.__name = name

        print(f"{self.__name} родился!")

In [ ]:
SampleClass.__dict__

In [ ]:
SampleClass(name="abc").__name

In [ ]:
SampleClass.__name

Эх, никак не можем получить доступ. Неужели мы обеспечили приватнось? Конечно же нет:

In [ ]:
SampleClass(name="abc")._SampleClass__name

А вот и достали нашу скрытую переменную. "Приватные" переменные можно вытащить из класса или инстанса класса посредством следующего нейминга: _ + `__class__` + private_attribute_name

<div style="
    background-color: #8B0000;
    padding: 15px;
    border: 2px dashed #ba0606;
    border-radius: 5px;
    text-align: center;
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 20px;
">
<span style="color: white; font-weight: bold;">
        Никогда не используйте двойное подчеркивание в нейминге атрибутов.
    </span>
</div>

### Protected

Вместо двойного подчеркивание разработчики договорились использовать одно подчеркивание - общая конвенция не трогать атрибуты, имя которых начинается с подчеркивания. Так и обеспечивается якобы приватность атрибутов в питоне.

In [ ]:
class SampleClass:
    def __init__(self, name: str) -> Self:
        self.name = name

        print(f"{self.name} родился!")

    def get_name(self):
        return self._name

    def set_name(self, name: str):
        raise ValueError("unsettable")

    _name = property(fget=get_name, fset=set_name)

In [ ]:
a = SampleClass(name="a")
a.name = "b"

In [ ]:
SampleClass(name="abc")._name

## Dataclass

<div style="
    background-color: #44944A;
    padding: 15px;
    border-radius: 10px;
    border: 1px solid #fbfbfbff;
    text-align: center;
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 15px;
">
    <span style="color: white; font-weight: bold;">
        dataclass - декоратор, модифицирующий поведение класса.
    </span>
</div>

Иногда нам нужен класс, который по сути представляет из себя структуру: в некоторых языках программирования есть отдельная сущность структура, которая представляет из себя что-то меньшее нежели класс, чаще всего это некое объединение множества полей в одно логическое имя.

Например, мы хотим написать класс, который просто хранит информацию о тесте:

In [ ]:
from dataclasses import dataclass


@dataclass
class Case:
    name: str
    given: int
    expected: int


class CaseClass:
    def __init__(self, name: str, given: int, expected: int) -> Self:
        self.name = name
        self.given = given
        self.expected = expected

    def __eq__(self, other: Self) -> bool:
        return (self.name, self.given, self.expected) == (
            other.name,
            other.given,
            other.expected,
        )

    def __repr__(self) -> str:
        return f"Case(name='{self.name}', given='{self.given}', expected='{self.expected}')"


c = Case("test_name", 2, 10)
c

In [ ]:
set(dir(Case)) - set(dir(CaseClass))

Данный декоратор добавил нашему классу некоторые атрибуты.

Можно конфигурировать датаклассы:

In [ ]:
help(dataclass)

In [ ]:
from dataclasses import dataclass, field


@dataclass(init=True, repr=True, eq=True, order=True, unsafe_hash=True, frozen=True)
class Case:
    name: str
    given: int
    expected: object
    s: int = field(default=10)


c = Case("test_name", 2, 1)
print(c)

Например, из-за флага frozen нельзя изменять инстанс датакласса:

In [ ]:
c.given = 3

## Enum

Еще один очень полезный тип классов - перечисление (от слова enumerate). Данный тип используется когда мы хотим завести тип с несколькими фиксированными значениями, например, тип операционной системы:

Чтобы сделать класс Enum'ом нужно наследовать его от `Enum`.

In [ ]:
from enum import Enum


class DeviceType(Enum):
    ANDROID = "android"
    WINDOWS = "windows"
    IOS = "ios"
    LINUX = "linux"
    MACOS = "macos"
    OTHER = "other"

In [ ]:
type(DeviceType.MACOS)

In [ ]:
d1 = DeviceType.MACOS
d2 = DeviceType.MACOS
d1 == d2, d1 is d2

Заметим, что значения в enum - всегда создаются только один раз.

In [ ]:
DeviceType.MACOS.name, DeviceType.MACOS.value

In [ ]:
{DeviceType.MACOS: 1, DeviceType.WINDOWS: 2}